In [1]:
#setting up frozen lake environment
import numpy as np
import gymnasium as gym
import random
import time
from IPython.display import clear_output

In [2]:
#Creating The Environment
env = gym.make("FrozenLake-v1", render_mode="human")

In [3]:
#the number of rows in the table is equivalent to the size of the state space in the environment, 
#and the number of columns is equivalent to the size of the action space
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

#create Q-table, and initialize all the Q-values to zero for each state-action pair.
#build the Q-table and fill it with zeros
q_table = np.zeros((state_space_size, action_space_size))
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [4]:
num_episodes = 10000 # total number of episodes we want the agent to play during training
max_steps_per_episode = 100 #maximum number of steps that our agent is allowed to take within a single episode

learning_rate = 0.1
discount_rate = 0.99

#exploration-exploitation trade-off (epsilon-greedy policy)
exploration_rate = 1

#max and min are just bounds to how large or small our exploration rate can be
max_exploration_rate =1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

In [ ]:
rewards_all_episodes = []

#Q-learning algorithm
for episode in range(num_episodes):
    state = env.reset()[0]
    
    done = False# status of the episode
    rewards_current_episode = 0
    for step in range(max_steps_per_episode):
        #Exploration - exploitation trde off
        exploration_rate_threshold = random.uniform(0,1);
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:]) #exploitation
        else:
            action = env.action_space.sample() #exploration
        new_state, reward, done, truncated, info = env.step(action)
        
        #update Q-table for Q(s,a)
        q_table[state,action] = q_table[state,action] * (1-learning_rate) + \
            learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        state = new_state
        rewards_current_episode += reward
        
        if done == True:
            break
    
    #Exploration rate decay
    exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate * episode)
    
    rewards_all_episodes.append(rewards_current_episode)
    
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes/1000)
count = 1000
print("Average reward per thousand episodes\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ",str(sum(r/1000)))
    count+= 1000

#Print updated Q-table
print("\n\nQtable")
print(q_table)

In [ ]:
for episode in range(3):
    state = env.reset()[0]
    done = False
    print("*****EPISODE ", episode+1, "*****\n\n\n\n")
    time.sleep(1)
    for step in range(max_steps_per_episode):        
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)
        
        action = np.argmax(q_table[state,:])        
        new_state, reward, done, truncated, info = env.step(action)
        
        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1:
                print("****You reached the goal!****")
                time.sleep(3)
            else:
                print("****You fell through a hole!****")
                time.sleep(3)
            clear_output(wait=True)
            break
        state = new_state
env.close()